In [ ]:
# Cell 1: Import libraries
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

In [ ]:

df1=pd.read_csv("picks_bans_2016.csv")
df2=pd.read_csv("picks_bans_2017.csv")
df3=pd.read_csv("picks_bans_2018.csv")
df4=pd.read_csv("picks_bans_2019.csv")
df5=pd.read_csv("picks_bans_2020.csv")
df6=pd.read_csv("picks_bans_2021.csv")
df7=pd.read_csv("picks_bans_2022.csv")
df8=pd.read_csv("picks_bans_2023.csv")
df9=pd.read_csv("picks_bans_2024.csv")
df10=pd.read_csv("picks_bans_2025Jan.csv")
df11=pd.read_csv("picks_bans_2025Feb.csv")
df12=pd.read_csv("picks_bans_2025Mar.csv")
df13=pd.read_csv("picks_bans_2025Apr.csv")
combined_df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13], ignore_index=True)
df14=pd.read_csv("dota2_hero_stats.csv")



combined_df.head()
df14.head()

In [ ]:

picks_only_df = combined_df[combined_df['is_pick'] == True]


hero_id_col_stats = 'id' if 'id' in df14.columns else 'id'
hero_id_col_picks = 'hero_id'

merged_df = pd.merge(
    picks_only_df,  # Use the filtered dataframe here
    df14,
    left_on=hero_id_col_picks,
    right_on=hero_id_col_stats,
    how='left'
)

# Check if any picks didn't match with hero stats
missing_matches = merged_df[merged_df[hero_id_col_stats].isna()].shape[0]
print(f"Records without matching hero stats: {missing_matches}")

# Save the combined dataset
merged_df.to_csv('dota2_complete_dataset.csv', index=False)
print(f"Successfully merged data into 'dota2_final_dataset.csv'")
print(f"Combined dataset has {merged_df.shape[0]} rows and {merged_df.shape[1]} columns")

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split # Make sure this is imported

def prepare_hero_recommendation_data(df):
    # List of all numeric hero stats to include
    hero_stat_columns = [
        'base_health', 'base_health_regen', 'base_mana', 'base_mana_regen',
        'base_armor', 'base_mr', 'base_attack_min', 'base_attack_max',
        'base_str', 'base_agi', 'base_int', 'str_gain', 'agi_gain', 'int_gain',
        'attack_range', 'attack_rate', 'base_attack_time',
        'attack_point', 'move_speed'
    ]

    # Convert stat columns to values
    for col in hero_stat_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        else:
            print(f"Warning: Stat column '{col}' not found in DataFrame.")

    # # Remove row if Nan values appear, trying to debug the NaN issue
    df['hero_id'] = pd.to_numeric(df['hero_id'], errors='coerce')
    df.dropna(subset=['hero_id'], inplace=True)
    df['hero_id'] = df['hero_id'].astype(int)



    # Creating a dictionary when processing data, easier data access
    hero_info = {}
    unique_heroes_df = df.drop_duplicates('hero_id').set_index('hero_id')

    for hero_id, row in unique_heroes_df.iterrows():
        hero_data = {
            'primary_attr': row.get('primary_attr', ''),
            'roles': row.get('roles', '')
        }
        # Add all numeric stats, filling NaNs with 0.0
        for stat in hero_stat_columns:
            if stat in row:

                stat_value = row[stat]
                # Fill NaN with 0.0
                hero_data[stat] = 0.0 if pd.isna(stat_value) else float(stat_value)
            else:
                 hero_data[stat] = 0.0 # Default if column doesn't exist

        hero_info[hero_id] = hero_data


    # Create attribute mapping
    attr_to_idx = {'str': 0, 'agi': 1, 'int': 2, 'all': 3}

    ## Extract roles
    all_roles = set()
    for hero_data in hero_info.values():
        roles = hero_data.get('roles', '')
        if isinstance(roles, str):
            for role in roles.split(','):
                clean_role = role.strip()
                if clean_role:
                    all_roles.add(clean_role)
    role_to_idx = {role: idx for idx, role in enumerate(sorted(all_roles))}
    num_roles = len(role_to_idx)
    print(f"Found {num_roles} unique roles")
    print(f"Collected stats for {len(hero_info)} heroes")


    # Data containers
    features_heroes = []
    features_attrs = []
    features_roles = []
    features_stats = []
    targets_next_heroes = []

    # Process matches
    print(f"Processing {df['match_id'].nunique()} matches...")
    match_groups = df.groupby('match_id')

    skipped_matches_count = 0
    processed_matches_count = 0

    for match_id, match_data in match_groups:
        #appends the different team picks into 2 list
        radiant_picks = match_data[match_data['team'] == 0]['hero_id'].tolist()
        dire_picks = match_data[match_data['team'] == 1]['hero_id'].tolist()

        # Skip matches with insufficient picks OR invalid hero IDs
        if len(radiant_picks) < 4 or len(dire_picks) < 2:
            skipped_matches_count += 1
            continue

        input_heroes = radiant_picks[:2] + dire_picks[:2]
        target_hero = radiant_picks[3] # Target is the 4th Radiant hero

        #Validate all heroes involved in this potential example
        all_involved_heroes = input_heroes + [target_hero]
        valid_match = True
        for hero_id in all_involved_heroes:
             # Check if hero_id is integer and exists in our processed hero_info
            if not isinstance(hero_id, (int, np.integer)) or hero_id not in hero_info:
                 #
                 valid_match = False
                 break
        if not valid_match:
            skipped_matches_count += 1
            continue
        #End Validation


        # If validation passed, extract features
        input_attrs = []
        input_roles = []
        input_stats_flat = []

        for hero_id in input_heroes:

            primary_attr = attr_to_idx.get(hero_info[hero_id]['primary_attr'], 0) # Default unknown attr

            roles_str = str(hero_info[hero_id].get('roles', ''))
            roles_list = roles_str.split(',') if roles_str else []
            primary_role_str = roles_list[0].strip() if roles_list else ''
            primary_role = role_to_idx.get(primary_role_str, num_roles) # Use num_roles as default index for unknown

            hero_stats_list = []
            for stat in hero_stat_columns:
                stat_value = hero_info[hero_id].get(stat, 0.0)
                hero_stats_list.append(float(stat_value))

            input_attrs.append(primary_attr)
            input_roles.append(primary_role)
            input_stats_flat.extend(hero_stats_list)


        # Final check on stats length before adding
        expected_stat_len = len(hero_stat_columns) * 4
        if len(input_stats_flat) != expected_stat_len:
             print(f"Warning: Final stat length mismatch for match {match_id}. Expected {expected_stat_len}, got {len(input_stats_flat)}. Skipping.")
             skipped_matches_count += 1
             continue


        # Add to datasets
        features_heroes.append(input_heroes)
        features_attrs.append(input_attrs)
        features_roles.append(input_roles)
        features_stats.append(input_stats_flat)
        targets_next_heroes.append(target_hero)
        processed_matches_count += 1


    print(f"Processed {processed_matches_count} examples.")
    print(f"Skipped {skipped_matches_count} matches due to insufficient picks or invalid data.")

    if not features_heroes:
         raise ValueError("No valid examples could be created. Check data quality and filtering logic.")

    # Convert to numpy arrays
    X_heroes = np.array(features_heroes)
    X_attrs = np.array(features_attrs)
    X_roles = np.array(features_roles)
    X_stats = np.array(features_stats, dtype=np.float32) # Specify dtype here
    y_heroes = np.array(targets_next_heroes)

    # --- Add Explicit Check for NaNs/Infs in NumPy array ---
    if np.isnan(X_stats).any():
        print("ERROR: NaNs found in X_stats NumPy array BEFORE splitting!")
        # Optionally find where: np.where(np.isnan(X_stats))
    if np.isinf(X_stats).any():
        print("ERROR: Infs found in X_stats NumPy array BEFORE splitting!")
    # --- End Check ---


    # Train-test split (ensure y_heroes has same length as X arrays)
    if len(X_stats) != len(y_heroes):
         raise ValueError(f"Feature arrays ({len(X_stats)}) and target array ({len(y_heroes)}) have different lengths before split.")

    (X_train_heroes, X_test_heroes,
     X_train_attrs, X_test_attrs,
     X_train_roles, X_test_roles,
     X_train_stats, X_test_stats,
     y_train_heroes, y_test_heroes) = train_test_split(
        X_heroes, X_attrs, X_roles, X_stats, y_heroes, # The arrays to split
        test_size=0.2,                                 # Proportion for the test set
        random_state=42                                # Ensures split is the same each time
     ) # Correct closing parenthesis here
    # Convert to tensors
    X_train_heroes_tensor = torch.tensor(X_train_heroes, dtype=torch.long)
    X_test_heroes_tensor = torch.tensor(X_test_heroes, dtype=torch.long)

    X_train_attrs_tensor = torch.tensor(X_train_attrs, dtype=torch.long)
    X_test_attrs_tensor = torch.tensor(X_test_attrs, dtype=torch.long)

    X_train_roles_tensor = torch.tensor(X_train_roles, dtype=torch.long)
    X_test_roles_tensor = torch.tensor(X_test_roles, dtype=torch.long)

    # Stats are already float32 numpy arrays
    X_train_stats_tensor = torch.from_numpy(X_train_stats)
    X_test_stats_tensor = torch.from_numpy(X_test_stats)

    y_train_heroes_tensor = torch.tensor(y_train_heroes, dtype=torch.long)
    y_test_heroes_tensor = torch.tensor(y_test_heroes, dtype=torch.long)

    if torch.isnan(X_train_stats_tensor).any():
        print("ERROR: NaNs found in X_train_stats_tensor!")

    if torch.isnan(X_test_stats_tensor).any():
        print("ERROR: NaNs found in X_test_stats_tensor!")



    # Create datasets
    train_dataset = TensorDataset(
        X_train_heroes_tensor, X_train_attrs_tensor, X_train_roles_tensor,
        X_train_stats_tensor, y_train_heroes_tensor
    )

    test_dataset = TensorDataset(
        X_test_heroes_tensor, X_test_attrs_tensor, X_test_roles_tensor,
        X_test_stats_tensor, y_test_heroes_tensor
    )

    # Create dataloaders
    batch_size = 64
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    print(f"Training set: {len(train_dataset)}, Test set: {len(test_dataset)}")

    # Determine num_heroes based on the MAX ID actually present + 1 (or use hero_info keys)
    # max_hero_id_present = max(hero_info.keys()) if hero_info else 0
    # num_heroes = max_hero_id_present + 1
    # Safer: Use max ID from the original DataFrame after cleaning
    max_hero_id_in_data = int(df['hero_id'].max())
    num_heroes = max_hero_id_in_data + 1 # Make sure index 0 is handled if used (e.g., padding)
    print(f"Determined num_heroes for embedding: {num_heroes} (based on max hero_id {max_hero_id_in_data})")

    # Define default index for unknown roles
    unknown_role_idx = num_roles # Assign the next available index
    role_to_idx['<UNK_ROLE>'] = unknown_role_idx
    num_roles += 1


    # Return necessary information
    return train_loader, test_loader, {
        'num_heroes': num_heroes,
        'num_attributes': len(attr_to_idx),
        'num_roles': num_roles, # Updated count including UNK
        'attr_mapping': attr_to_idx,
        'role_mapping': role_to_idx, # Updated mapping
        'hero_info': hero_info,
        'stat_columns': hero_stat_columns,
        'stat_dim': len(hero_stat_columns) * 4, # Total dimension of flattened hero stats (4 heroes)
    }

In [ ]:
# Define the hero recommendation model with hero stats
class HeroRecommender(nn.Module):
    def __init__(self, num_heroes, num_roles, num_attributes, stat_dim=0,
                 hero_embedding_dim=128, role_embedding_dim=32, attr_embedding_dim=16):
        super(HeroRecommender, self).__init__()

        # Hero embeddings
        self.hero_embeddings = nn.Embedding(num_heroes, hero_embedding_dim, padding_idx=0)

        # Role embeddings
        self.role_embeddings = nn.Embedding(num_roles, role_embedding_dim)

        # Attribute embeddings
        self.attr_embeddings = nn.Embedding(num_attributes, attr_embedding_dim)


        # Stats processor (if stats are provided)
        self.use_stats = stat_dim > 0

        #Stat_dim=78, therefore 128 is sufficient
        #Potentially changing the hidden layers in the future, if time permits
        if self.use_stats:
            self.stats_processor = nn.Sequential(
                nn.Linear(stat_dim, 128),
                nn.BatchNorm1d(128),
                nn.LeakyReLU(),
                nn.Dropout(0.4),

                nn.Linear(128, 256),
                nn.BatchNorm1d(256),
                nn.LeakyReLU(),
                nn.Dropout(0.4),

                nn.Linear(256, 512),
                nn.BatchNorm1d(512),
                nn.LeakyReLU(),
                nn.Dropout(0.4),

                nn.Linear(512, 256),
                nn.BatchNorm1d(256),
                nn.LeakyReLU(),
                nn.Dropout(0.4),

                nn.Linear(256, 128),
                nn.BatchNorm1d(128),
                nn.LeakyReLU(),
                nn.Dropout(0.4),
            )
            stats_output_dim = 128
        else:
            stats_output_dim = 0


        # Total embedding dimension per hero
        single_hero_dim = hero_embedding_dim + role_embedding_dim + attr_embedding_dim

        # Total input dimension (4 heroes + processed stats if available)
        total_input_dim = (single_hero_dim * 4) + stats_output_dim
        print

        # Hidden dimensions
        hidden1 = total_input_dim*2
        hidden2 = total_input_dim*4
        hidden3 = total_input_dim*8
        hidden4 = total_input_dim*16
        hidden5 = total_input_dim*32

        # Encoder network
        self.encoder = nn.Sequential(
            nn.Linear(total_input_dim,hidden1),
            nn.BatchNorm1d(hidden1),
            nn.LeakyReLU(),
            nn.Dropout(0.55),

            nn.Linear(hidden1, hidden2),
            nn.BatchNorm1d(hidden2),
            nn.LeakyReLU(),
            nn.Dropout(0.55),

            nn.Linear(hidden2, hidden3),
            nn.BatchNorm1d(hidden3),
            nn.LeakyReLU(),
            nn.Dropout(0.55),

            nn.Linear(hidden3, hidden4),
            nn.BatchNorm1d(hidden4),
            nn.LeakyReLU(),
            nn.Dropout(0.55),

            nn.Linear(hidden4, hidden3),
            nn.BatchNorm1d(hidden3),
            nn.LeakyReLU(),
            nn.Dropout(0.55),

            nn.Linear(hidden3, hidden2),
            nn.BatchNorm1d(hidden2),
            nn.LeakyReLU(),
            nn.Dropout(0.55),

            nn.Linear(hidden2, hidden1),
            nn.BatchNorm1d(hidden1),
            nn.LeakyReLU(),
            nn.Dropout(0.55),
        )

        # Output layer
        self.hero_decoder = nn.Linear(hidden1, num_heroes)

    def forward(self, hero_ids, attr_ids, role_ids, hero_stats=None):
        # Process each hero
        hero_embeds = []

        for i in range(4):  # Process 4 heroes
            # Get hero embedding
            hero_embed = self.hero_embeddings(hero_ids[:, i])


            # Get attribute embedding
            attr_embed = self.attr_embeddings(attr_ids[:, i])


            # Get role embedding
            role_embed = self.role_embeddings(role_ids[:, i])


            # Combine embeddings for this hero
            hero_combined = torch.cat([hero_embed, attr_embed, role_embed], dim=1)
            hero_embeds.append(hero_combined)

        #To process stats
        if self.use_stats and hero_stats is not None:
            processed_stats = self.stats_processor(hero_stats)
            # Combine all hero embeddings and processed stats
            combined_input = torch.cat(hero_embeds + [processed_stats], dim=1)

        else:
            # When stats are enabled but not provided, create zeros tensor
            if self.use_stats:
                batch_size = hero_ids.shape[0]
                dummy_stats = torch.zeros(batch_size, 128, device=hero_ids.device)
                combined_input = torch.cat(hero_embeds + [dummy_stats], dim=1)
            else:
                combined_input = torch.cat(hero_embeds, dim=1)

        # Encode draft state
        encoded = self.encoder(combined_input)

        # Predict next hero
        hero_scores = self.hero_decoder(encoded)

        # Mask out already picked heroes
        hero_mask = torch.zeros_like(hero_scores, dtype=torch.bool)
        for i in range(hero_scores.size(0)):
            for j in range(4):
                hero_id = hero_ids[i, j].item()
                if hero_id > 0:  # Skip padding
                    hero_mask[i, hero_id] = True

            # Also mask padding index
            hero_mask[i, 0] = True

        hero_scores = hero_scores.masked_fill(hero_mask, -100)


        return hero_scores


model = HeroRecommender(
    num_heroes=feature_maps['num_heroes'],
    num_roles=feature_maps['num_roles'],
    num_attributes=feature_maps['num_attributes'],
    stat_dim=stat_dim
).to(device)

print(model)
print( )

In [ ]:
# Define the training function
def train_hero_recommender(model, train_loader, test_loader, num_epochs=20, lr=0.0001):
    # Set device
    device = next(model.parameters()).device

    # Criterion and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)




    # Metrics storage
    train_losses = []
    test_losses = []
    train_accs = []
    test_accs = []
    train_top10_accs = []
    test_top10_accs = []

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        correct_top1 = 0
        correct_top10 = 0
        total_samples = 0

        for batch in train_loader:
            # Unpack the batch based on its structure
            if len(batch) == 4:
                hero_ids, attr_ids, role_ids, targets = batch
                # Model doesn't use hero_stats, so don't need to pass it
            elif len(batch) == 5:
                hero_ids, attr_ids, role_ids, hero_stats, targets = batch
                # ignore hero_stats to match model's forward signature
            else:
                continue

            # Move to device
            hero_ids = hero_ids.to(device)
            attr_ids = attr_ids.to(device)
            role_ids = role_ids.to(device)
            targets = targets.to(device)

            if len(targets.shape) > 1:
                targets = targets.squeeze()




            # Forward pass - only using the parameters the model accept
            optimizer.zero_grad()
            if len(batch) == 4:
                # Use the tensors that were already moved to device
                hero_scores = model(hero_ids, attr_ids, role_ids)
            elif len(batch) == 5:
                # Move hero_stats to device and then use all tensors
                hero_stats = hero_stats.to(device)
                hero_scores = model(hero_ids, attr_ids, role_ids, hero_stats)
            # Calculate loss
            loss = criterion(hero_scores, targets)



            # Backward pass
            loss.backward()

            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()

            total_loss += loss.item()

            # Get top-1 and top-10 predictions
            _, top1 = torch.max(hero_scores, 1)
            _, top10 = torch.topk(hero_scores, k=10, dim=1)

            # Convert tensors to numpy for easier comparison
            top10_np = top10.cpu().numpy()
            targets_np = targets.cpu().numpy()

            # Count top-1 correct predictions
            correct_top1 += (top1 == targets).sum().item()

            # Count top-10 correct predictions
            batch_size = targets.size(0)
            for i in range(batch_size):
                if targets_np[i] in top10_np[i]:
                    correct_top10 += 1

            total_samples += batch_size

        # Test phase
        model.eval()
        test_loss = 0
        test_correct_top1 = 0
        test_correct_top10 = 0
        test_total_samples = 0

        with torch.no_grad():
            for batch in test_loader:
                # Unpack the batch based on its structure
                if len(batch) == 4:
                    hero_ids, attr_ids, role_ids, targets = batch
                    # Model doesn't use hero_stats, so we don't need to pass it
                elif len(batch) == 5:
                    hero_ids, attr_ids, role_ids, hero_stats, targets = batch
                    # We'll ignore hero_stats to match model's forward signature
                else:
                    continue

                # Move to device
                hero_ids = hero_ids.to(device)
                attr_ids = attr_ids.to(device)
                role_ids = role_ids.to(device)
                targets = targets.to(device)

                if len(targets.shape) > 1:
                    targets = targets.squeeze()


                # Forward pass - only using the parameters the model accepts
                hero_scores = model(hero_ids, attr_ids, role_ids)

                # Calculate loss
                loss = criterion(hero_scores, targets)

                # Skip NaN loss
                if torch.isnan(loss):
                    continue

                test_loss += loss.item()

                # Get top-1 and top-10 predictions
                _, top1 = torch.max(hero_scores, 1)
                _, top10 = torch.topk(hero_scores, k=10, dim=1)

                # Convert tensors to numpy for easier comparison
                top10_np = top10.cpu().numpy()
                targets_np = targets.cpu().numpy()

                # Count top-1 correct predictions
                test_correct_top1 += (top1 == targets).sum().item()

                # Count top-10 correct predictions
                batch_size = targets.size(0)
                for i in range(batch_size):
                    if targets_np[i] in top10_np[i]:
                        test_correct_top10 += 1

                test_total_samples += batch_size

        # Calculate metrics
        train_loss = total_loss / len(train_loader)
        train_acc_top1 = correct_top1 / total_samples if total_samples > 0 else 0
        train_acc_top10 = correct_top10 / total_samples if total_samples > 0 else 0

        test_loss = test_loss / len(test_loader)
        test_acc_top1 = test_correct_top1 / test_total_samples if test_total_samples > 0 else 0
        test_acc_top10 = test_correct_top10 / test_total_samples if test_total_samples > 0 else 0

        # Store metrics
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        train_accs.append(train_acc_top1)
        test_accs.append(test_acc_top1)
        train_top10_accs.append(train_acc_top10)
        test_top10_accs.append(test_acc_top10)

        # Update learning rate
        scheduler.step(test_loss)


        # Print metrics
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"Training Loss: {train_loss:.4f}, Top-1 Acc: {train_acc_top1:.4f}, Top-10 Acc: {train_acc_top10:.4f}")
        print(f"Validation Loss: {test_loss:.4f}, Top-1 Acc: {test_acc_top1:.4f}, Top-10 Acc: {test_acc_top10:.4f}")
        print("-" * 50)



    return model, train_losses, test_losses, train_accs, test_accs, train_top10_accs, test_top10_accs

In [ ]:
num_epochs = 50  # Adjust as needed
model, train_losses, test_losses, train_accs, test_accs, train_top10_accs, test_top10_accs = train_hero_recommender(
    model, train_loader, test_loader, num_epochs=num_epochs
)

In [ ]:
# Visualize training progress
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 5))

# Plot losses
plt.subplot(1, 3, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss Value')
plt.legend()

# Plot top-1 accuracy
plt.subplot(1, 3, 2)
plt.plot(train_accs, label='Train Top-1 Acc')
plt.plot(test_accs, label='Test Top-1 Acc')
plt.title('Top-1 Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot top-10 accuracy
plt.subplot(1, 3, 3)
plt.plot(train_top10_accs, label='Train Top-10 Acc')
plt.plot(test_top10_accs, label='Test Top-10 Acc')
plt.title('Top-10 Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.savefig('training_metrics.png')
plt.show()

In [ ]:
import torch


example_input = (
    torch.tensor([[1, 2, 3, 4]], dtype=torch.long),  # Example hero IDs
    torch.tensor([[0, 1, 2, 0]], dtype=torch.long),  # Example attribute IDs
    torch.tensor([[5, 3, 1, 2]], dtype=torch.long),  # Example role IDs
    torch.randn(1, 76)  # Example hero stats
)

model_save_path = '/content/drive/MyDrive/hero_recommender_model5th.pth'
torch.save(model.state_dict(), model_save_path)  # Save model state dict to a file
